In [37]:
import pandas as pd
import sklearn.model_selection as ms
import sklearn
from sklearn.linear_model import LinearRegression

In [103]:
df = pd.read_csv("workable_df.csv")

In [104]:
df.gender = df['gender'].apply(str)
df.user_type = df['user_type'].apply(str)
df.start_station_id = df['start_station_id'].apply(str)

In [105]:
binary = []
for i in range(len(df)):
    if df['trip_duration'][i] < 3600:
        binary.append('0')
    else:
        binary.append('1')

In [108]:
df['over_1_hr'] = binary

In [109]:
df1 = df.drop('start_hour_sq', axis = 1)
df1 = df1.drop('start_month_sq', axis = 1)
df1 = pd.get_dummies(df1,drop_first=True)
df1 = df1.drop('median_sale_price', axis = 1)
df1 = df1.dropna(axis=0)
df1.dtypes

trip_duration                      int64
start_month                        int64
start_hour                         int64
birth_year                         int64
total_precipitation_inches       float64
average_temperature_farenheit    float64
total_snowfall_inches            float64
median_rental_price              float64
start_station_id_119               uint8
start_station_id_120               uint8
start_station_id_127               uint8
start_station_id_128               uint8
start_station_id_143               uint8
start_station_id_144               uint8
start_station_id_146               uint8
start_station_id_150               uint8
start_station_id_151               uint8
start_station_id_152               uint8
start_station_id_157               uint8
start_station_id_161               uint8
start_station_id_164               uint8
start_station_id_167               uint8
start_station_id_168               uint8
start_station_id_173               uint8
start_station_id

In [116]:
from sklearn.tree import DecisionTreeClassifier 

X = df1.drop('trip_duration', axis = 1)
X = X.drop('over_1_hr_1', axis = 1)
y = df1['over_1_hr_1']
X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.2, random_state=292)

# BAD classifier = DecisionTreeClassifier(random_state = 58, class_weight = 'balanced')  
classifier = DecisionTreeClassifier(random_state = 58)  
# fit the regressor with X and Y data 
classifier.fit(X_train, y_train) 

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=58,
            splitter='best')

In [117]:
#Confusion matrix. This model is bad. 
ypred = classifier.predict(X_test)
from sklearn import metrics
sklearn.metrics.confusion_matrix(y_test, ypred)

array([[114351,   1088],
       [   881,     90]])

In [118]:
X = df1.drop('trip_duration', axis = 1)
y = df1['trip_duration']
X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.2, random_state=292)

In [124]:
from sklearn.tree import DecisionTreeRegressor  

#Only for < 3600 seconds. 
df = df[df['trip_duration']<3600]

# create a regressor object 
regressor = DecisionTreeRegressor(random_state = 5, max_depth = 6)  
  
# fit the regressor with X and Y data 
regressor.fit(X_train, y_train) 

DecisionTreeRegressor(criterion='mse', max_depth=6, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=5, splitter='best')

In [125]:
y_pred = regressor.predict(X_test)

In [126]:
#Test MAE
sum(abs(y_pred-y_test)) / len(y_test)

603.3350794442416

In [127]:
#Train MAE #Slightly better fit, not necessarily overfit. 
sum(abs(regressor.predict(X_train)-y_train)) / len(y_train)

553.7402600082179